In [0]:
%pip install openai unidecode gspread==5.12.4 tiktoken mlflow
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from functools import reduce
from typing import *
import tiktoken


from general_config import *
from ml_tracker import MLTracker
from in_game_localizer import InGameLocalizer
#from InGame_Config import * 

In [0]:
%run "./authenticationScript"

In [0]:
#TEST_ACCOUNT = "gspread-access@ds-localization.iam.gserviceaccount.com"

In [0]:
#TODO update authenticatinon script and secrets!!
gsheet_client = get_gspread_client_from_secret_old()
gpt_client = get_model_client() 

In [0]:
#gsheet_client.

In [0]:
#import datetime
#datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


In [0]:
request =  {"RowFingerprint":"testing_v1",
                    "Timestamp":"2025-08-26 20:47:27",
                    "SubmitterEmail":"krista@jamcity.com",
                    "DueDate":"8/27/25",
                    'LocType':"InGame", 
                    'Game':"Panda Pop", 
                    'TargetLanguages':"", 
                    "URL":"https://docs.google.com/spreadsheets/d/1XlJczcGUzgKhMZCj0D-QiTxFYFleWlB4WPEXi0LPElk/edit?gid=0#gid=0",
                    "QAFlag":""}


cfg = {
    "input": {"required_tabs": ["input","output"], "input_headers": [], "output_headers": []},
    "char_limit_policy": "loose"
}

In [0]:
#"testing_v0"

In [0]:
localizer = InGameLocalizer(request = request, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)

In [0]:
#localizer.__dict__

In [0]:
#request['URL']

In [0]:
localizer.gc.open_by_url(request['URL'])

In [0]:
localizer.validate_inputs()

In [0]:
inputs = localizer.load_inputs()

preprocessed = localizer.preprocess(inputs)

In [0]:
prompts = localizer.build_prompts(preprocessed)

In [0]:
raw_results = localizer.translate(prompts)

In [0]:
#For testing
#localizer._parse_model_json_block(raw_results[0])

In [0]:
post_processed = localizer.postprocess(raw_results)


In [0]:
localizer.write_outputs(post_processed)

In [0]:
input = localizer.sh.worksheet('input').get_all_values()
input[0]

In [0]:
output = localizer.sh.worksheet('output').get_all_values()
output[0]

In [0]:
localizer.lang_trackers